# Projeto 1 - Ciência dos Dados

Nome: André Brito

Nome: Gabriela Albuquerque

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [28]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [29]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/andrebrito/InsperCd2021.2/Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [30]:
def cleanup(text):
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed.strip()

In [31]:
filename = 'iPhone 13.xlsx'

In [32]:
train = pd.read_excel(filename)
train.Treinamento = train['Treinamento'].apply(cleanup)
train.head(5)

,Treinamento,Classificador
0,alguém me da um iphone 13 to precisando 🥲,1
1,@vicmoura14 nem eu afinal aquele iphone 13 n v...,0
2,os outros 4 mentiram 😂 https//tco/vleieec9vm,0
3,@dosievil um macbook air 13 2 iphone 11 pro ma...,0
4,já tá curioso pelo iphone 13 https//tco/wk8ibo...,1


In [33]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.Teste = test['Teste'].apply(cleanup)
test.head(5)

,Teste,Classificador
0,@felipeneto o iphone 13 sem entrada de carrega...,1
1,iphone 13 n vir com entrada pra carregador e o...,1
2,iphone 13 nem lançou e eu já tô querendo comprar,1
3,iphone 13 vir sem entrada pra carregador te ob...,1
4,@kayanes7 @lara_ray002 @anonymouscrec poise ma...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto é o iPhone 13, que tem o lançamento previsto para o mês de setembro de 2021. Foram considerados relevantes tweets que falam algo sobre o aparelho mas sem nenhum tom de piada ou meme.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [34]:
# Separar os Tweets em categorias
iPhone13_relevante = train[train['Classificador'] == 1]
iPhone13_nao_relevante = train[train['Classificador'] == 0]

In [35]:
# Salvamento dos tweets como strings
Tweets_relevantes_Text = ""
for i in iPhone13_relevante['Treinamento']:
  Tweets_relevantes_Text += str(i)


Tweets_nao_relevantes_Text = ""
for i in iPhone13_nao_relevante['Treinamento']:
  Tweets_nao_relevantes_Text += str(i)

todos_tweets_relevantes = Tweets_relevantes_Text.split()
todos_tweets_irrelevantes = Tweets_nao_relevantes_Text.split()

serie_tweets_relevantes = pd.Series(todos_tweets_relevantes)
serie_tweets_irrelevantes = pd.Series(todos_tweets_irrelevantes)


In [36]:
tabela_relevantes = serie_tweets_relevantes.value_counts()
tabela_relevantes

iphone                             167
13                                 162
o                                  128
de                                  83
que                                 74
                                  ... 
“revolucionar”                       1
https//tco/ui6ywtr4gv                1
descobriam                           1
disponível                           1
https//tco/v86e1kbceolançamento      1
Length: 1095, dtype: int64

In [37]:
tabela_relevantes_relativa = serie_tweets_relevantes.value_counts(True)
tabela_relevantes_relativa

iphone                             0.051496
13                                 0.049954
o                                  0.039470
de                                 0.025594
que                                0.022818
                                     ...   
“revolucionar”                     0.000308
https//tco/ui6ywtr4gv              0.000308
descobriam                         0.000308
disponível                         0.000308
https//tco/v86e1kbceolançamento    0.000308
Length: 1095, dtype: float64

In [38]:
tabela_irrelevantes = serie_tweets_irrelevantes.value_counts()
tabela_irrelevantes

o                         106
iphone                    104
13                         85
do                         54
e                          41
                         ... 
oq                          1
https//tco/9fkh8dqtmja      1
zoada                       1
umas                        1
rico                        1
Length: 667, dtype: int64

In [39]:
tabela_irrelevantes_relativa = serie_tweets_irrelevantes.value_counts(True)
tabela_irrelevantes_relativa

o                         0.055381
iphone                    0.054336
13                        0.044410
do                        0.028213
e                         0.021421
                            ...   
oq                        0.000522
https//tco/9fkh8dqtmja    0.000522
zoada                     0.000522
umas                      0.000522
rico                      0.000522
Length: 667, dtype: float64

In [40]:
tweets = Tweets_relevantes_Text + Tweets_nao_relevantes_Text

todos_tweets = tweets.split()

serie_todos_tweets = pd.Series(todos_tweets)


In [41]:
tabela_tweets = serie_todos_tweets.value_counts()
tabela_tweets

iphone                         271
13                             247
o                              234
de                             117
e                              107
                              ... 
apesar                           1
ponte                            1
comentados                       1
https//tco/9einvlxbdbboatos      1
bozomeu                          1
Length: 1536, dtype: int64

In [42]:
tabela_tweets_relativa = serie_todos_tweets.value_counts(True)
tabela_tweets_relativa

iphone                         0.052560
13                             0.047905
o                              0.045384
de                             0.022692
e                              0.020753
                                 ...   
apesar                         0.000194
ponte                          0.000194
comentados                     0.000194
https//tco/9einvlxbdbboatos    0.000194
bozomeu                        0.000194
Length: 1536, dtype: float64

In [43]:
tabela_tweets

iphone                         271
13                             247
o                              234
de                             117
e                              107
                              ... 
apesar                           1
ponte                            1
comentados                       1
https//tco/9einvlxbdbboatos      1
bozomeu                          1
Length: 1536, dtype: int64

In [48]:
def suavizacao_laplace(palavra, classificao):
    # Classificação para irrelevante
    if classificao == 0:
      if palavra in tabela_irrelevantes:
        return (tabela_irrelevantes[palavra] + 1) / (tabela_irrelevantes.sum() + len(tabela_tweets))
      else:
        return 1 / (tabela_irrelevantes.sum() + len(tabela_tweets))
      
    # Classificação para relevante
    if classificao == 1:
      if palavra in tabela_relevantes:
        return (tabela_relevantes[palavra] + 1) / (tabela_relevantes.sum() + len(tabela_tweets))
      else:
        return 1 / (tabela_relevantes.sum() + len(tabela_tweets))
      

In [45]:
def calcula_probabilidade(tweet):
  probRelevante_laplace = []
  probIrrelevante_laplace = []
  probRelevante = 1
  probIrrelevante = 1

  for i in tweet:
    probIrrelevante_laplace.append(suavizacao_laplace(i, 0))
    probRelevante_laplace.append(suavizacao_laplace(i, 1))

  for i in probIrrelevante_laplace:
    probIrrelevante *= i
  
  for i in probRelevante_laplace:
    probRelevante *= i

  if probRelevante > probIrrelevante:
    return 1
  else:
    return 0

  

In [46]:
def classificador(tweets):
  lista_tweets = []
  lista_tweets_prob = []

  for i in tweets['Teste']:
    lista_tweets.append(i.lower())
  
  for i in lista_tweets:
    tweets_splitados = i.strip().split()
    lista_tweets_prob.append(calcula_probabilidade(tweets_splitados))

  return lista_tweets_prob

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [47]:
resultado = classificador(test)

test['Classificador Naive Bias'] = resultado
test.head(30)

,Teste,Classificador,Classificador Naive Bias
0,@felipeneto o iphone 13 sem entrada de carrega...,1,1
1,iphone 13 n vir com entrada pra carregador e o...,1,1
2,iphone 13 nem lançou e eu já tô querendo comprar,1,1
3,iphone 13 vir sem entrada pra carregador te ob...,1,1
4,@kayanes7 @lara_ray002 @anonymouscrec poise ma...,0,1
5,ai vocês que são ricos e vão trocar de iphone ...,0,1
6,@jotaenii7 cnf iphone 13 vai chegar,1,1
7,@joaquinteixeira programação dia07vaisergigant...,0,0
8,comunicação por satélite do iphone 13 será exc...,1,1
9,@ruivo367 @_lasg @mariliamoscou e várias outra...,0,1


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**